In [1]:
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
import cv2

In [2]:
def NHE(img, n):
    
    # step length
    imgMin = img.min()
    imgMax = img.max()
    step = (imgMax - imgMin)/n
    
    for i in range(n):
        if i == 0:
            sub_bound = round(imgMin + step)
            simg = img.flatten().compress((img.flatten() <= sub_bound).flat)
            
            hist, bins = np.histogram(simg, sub_bound+1, [0, sub_bound])
            pdf = hist / np.prod(simg.size)
            cdf = pdf.cumsum()
            
            cdf = cdf *(sub_bound - imgMin) + imgMin
            continue;
        elif i == n-1:
            sub_bound = round(imgMax - step)
            simg = img.flatten().compress((img.flatten() > sub_bound).flat)
            
            hist, bins = np.histogram(simg, 256-sub_bound, [sub_bound+1, 256])
            pdf = hist / np.prod(simg.size)
            cdf2 = pdf.cumsum()
            
            cdf2 = cdf2 *(imgMax - (sub_bound+1)) + sub_bound+1
        else:
            simg = img.flatten().compress((img.flatten() <= round(imgMin + (i+1)*step)).flat)
            simg = simg.compress(simg > round(imgMin + i*step))
            
            hist, bins = np.histogram(simg, round(imgMin + (i+1)*step) - round(imgMin + i*step), [round(imgMin + i*step)+1, round(imgMin + (i+1)*step)])
            pdf = hist / np.prod(simg.size)
            cdf2 = pdf.cumsum()
            
            cdf2 = cdf2 *(round(imgMin + (i+1)*step) - (round(imgMin + i*step)+1)) + round(imgMin + i*step)+1
        
        cdf = np.concatenate((cdf, cdf2))
    
    new_img = cdf[img.ravel()]
    img_eq = np.reshape(new_img, img.shape)
    img_eq = img_eq.astype('uint8')
    return img_eq

#     # getting two subimages
#         # Numpy compress() function returns the selected slices of an array along the given axis. 
#         # When working along a given axis, a slice along that axis is returned in output for each 
#         # index where condition evaluates to True.
#     img_l = img.flatten().compress((img.flatten() <= img_mean).flat)
#     img_u = img.flatten().compress((img.flatten() > img_mean).flat)
    
#     # cdf of low subimage
#     hist_l, bins_l = np.histogram(img_l, img_mean+1, [0, img_mean])
#     pdf_l = hist_l / np.prod(img_l.size)
#     cdf_l = pdf_l.cumsum()

#     # transform func of low
#     cdf_l = cdf_l *(img_mean - img.min()) + img.min()          
    
    
#     # cdf of upper subimage
#     hist_u, bins_u = np.histogram(img_u, 256-img_mean, [img_mean+1, 256])
#     pdf_u = hist_u / np.prod(img_u.size)
#     cdf_u = pdf_u.cumsum()

#     # transform func of upper
#     cdf_u = cdf_u *(img.max() - (int(img_mean) + 1)) + (int(img_mean) + 1)
    
#     cdf_new = np.concatenate((cdf_l, cdf_u))
#     new_img = cdf_new[img.ravel()]
#     img_eq = np.reshape(new_img, img.shape)
    
#     return img_eq

In [3]:
def draw_result(img_eq):
    # merge channels
    hsv_img = cv2.merge([H,S,img_eq])
    #convert to BGR channels
    new_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2BGR)
    return new_img

def plot_draw(new_img):
    hist2, bin2 = np.histogram(new_img, 256)
    pdf2 = hist2 / np.prod(img.shape)
    #plt.subplot(2, 1, 1)
    plt.plot(pdf2, color='r')
    plt.title("HE image")
    plt.xlim([0, 256])
    plt.ylim([0, pdf2.max()+0.005])    
    #plt.subplot(2, 1, 2) 
    

In [ ]:
# [sample01.jpg, sample02.jpeg, sample03.jpeg, sample04.jpeg, sample05.jpeg, sample06.jpg, sample07,jpg, sample08.jpg]
# read in image
imgStrList1 = ("sample01.jpg","sample02.jpeg","sample03.jpeg","sample04.jpeg","sample05.jpeg","sample06.jpg","sample07.jpg","sample08.jpg")
imgStrList2 = ("sample01_SwinIR.png","sample02_SwinIR.png","sample03_SwinIR.png","sample04_SwinIR.png","sample05_SwinIR.png","sample06_SwinIR.png","sample07_SwinIR.png","sample08_SwinIR.png") 
imgStrList = imgStrList1 + imgStrList2
colorCv2 = ("b", "g", "r")
#colorPlt = ("r", "g", "b")

for imgStr in imgStrList2:
    img = cv2.imread(imgStr) #彩色
#     imgGrey = cv2.imread(imgStr, 0) #灰度
#     imgHE = cv2.equalizeHist(imgGrey)
#     clahe = cv2.createCLAHE(clipLimit = 2.0, tileGridSize = (8, 8))
#     imgCLAHE = clahe.apply(imgGrey)
    
#     # histogram
#     for i, color in enumerate(colorCv2):
#         hist = cv2.calcHist([img], [i], None, [256], [0, 256])
#         plt.plot(hist, color = color)
#         plt.xlim([0, 256])

     # Converting BGR-format image to HSV-format image
    HSV_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    H,S,V = cv2.split(HSV_image)
    
#     # show HSV-format image
#     cv2.imshow('HSV Format image', HSV_image)
#     cv2.waitKey(0)

    img_eq = NHE(V,1)
    HE = draw_result(img_eq)
    plot_draw(HE)

    img_eq2 = NHE(V,2)
    BHE = draw_result(img_eq2)
    plot_draw(BHE)

    img_eq3 = NHE(V,3)
    THE = draw_result(img_eq3)
    plot_draw(THE)

    img_eq4 = NHE(V,4)
    FHE = draw_result(img_eq4)
    plot_draw(FHE)
    
    imgYUV = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
    
    img_eq11 = imgYUV.copy()
    img_eq22 = imgYUV.copy()
    img_eq33 = imgYUV.copy()
    img_eq44 = imgYUV.copy()
    
    img_eq11[:,:,0] = NHE(img_eq11[:,:,0],1)
    img_eq22[:,:,0] = NHE(img_eq22[:,:,0],2)
    img_eq33[:,:,0] = NHE(img_eq33[:,:,0],3)
    img_eq44[:,:,0] = NHE(img_eq44[:,:,0],4)
    
    HE1 = cv2.cvtColor(img_eq11, cv2.COLOR_YUV2BGR)
    BHE2 = cv2.cvtColor(img_eq22, cv2.COLOR_YUV2BGR)
    THE3 = cv2.cvtColor(img_eq33, cv2.COLOR_YUV2BGR)
    FHE4 = cv2.cvtColor(img_eq44, cv2.COLOR_YUV2BGR)

    cv2.imshow(imgStr,np.hstack((img,HE,BHE,THE,FHE)))
    cv2.imshow(imgStr + " YUV",np.hstack((img,HE1,BHE2,THE3,FHE4)))
    
cv2.waitKey(0) 
cv2.destroyAllWindows()